In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [ ]:
# Import data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
wetter = pd.read_csv('wetter.csv')
kiwo = pd.read_csv('kiwo.csv')

In [3]:
# Convert Dates
train['Datum'] = pd.to_datetime(train['Datum'])
test['Datum'] = pd.to_datetime(test['Datum'])
wetter['Datum'] = pd.to_datetime(wetter['Datum'])
kiwo['Datum'] = pd.to_datetime(kiwo['Datum'])

In [4]:
# Merge Train and Test
# We merge them to apply the same feature engineering to both
train['is_train'] = True
test['is_train'] = False
merged_data = pd.concat([train, test], sort=False)

In [5]:
# Merge External Data
merged_data = merged_data.merge(wetter, on='Datum', how='left')
merged_data = merged_data.merge(kiwo, on='Datum', how='left')

In [6]:
# Missing Value Imputation (Week 9)
# KielerWoche: NaNs mean it's not Kieler Woche -> fill with 0
merged_data['KielerWoche'] = merged_data['KielerWoche'].fillna(0)

In [7]:
# Weather: Forward fill then backward fill for simple imputation
merged_data['Bewoelkung'] = merged_data['Bewoelkung'].fillna(method='ffill').fillna(method='bfill')
merged_data['Temperatur'] = merged_data['Temperatur'].fillna(method='ffill').fillna(method='bfill')
merged_data['Windgeschwindigkeit'] = merged_data['Windgeschwindigkeit'].fillna(method='ffill').fillna(method='bfill')
# Wettercode: Fill with 0 (assuming 0 is "no significant weather" or unknown)
merged_data['Wettercode'] = merged_data['Wettercode'].fillna(0)

/tmp/ipykernel_23418/2792657794.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data['Bewoelkung'] = merged_data['Bewoelkung'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_23418/2792657794.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data['Temperatur'] = merged_data['Temperatur'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_23418/2792657794.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data['Windgeschwindigkeit'] = merged_data['Windgeschwindigkeit'].fillna(method='ffill').fillna(method='bfill')


In [8]:
# Feature Engineering (Week 4)
merged_data['Year'] = merged_data['Datum'].dt.year
merged_data['Month'] = merged_data['Datum'].dt.month
merged_data['Day'] = merged_data['Datum'].dt.day
merged_data['Weekday'] = merged_data['Datum'].dt.weekday # 0=Mon, 6=Sun
merged_data['IsWeekend'] = merged_data['Weekday'] >= 5

In [9]:
# One-Hot Encoding for Warengruppe (Categorical)
merged_data = pd.get_dummies(merged_data, columns=['Warengruppe'], prefix='Warengruppe', drop_first=False)

In [10]:
# Save merged data
merged_data.to_csv('merged_data.csv', index=False)